In [40]:
from import_src import import_src
import_src()

Append /home/v_user/khiempm/MLOpsVN/src to sys.path


In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import json
from plotly import express as px
import mlflow
import seaborn as sns
import time
import pickle

In [42]:
pd.set_option('display.max_columns', 500)

In [43]:
link_data_test_raw_32 = '../../data/captured_data/phase-3/prob-2/raw/'
link_data_test_raw_22 = '../../data/captured_data/phase-2/prob-2/raw/'
train_raw_data_32 = '../../data/raw_data/phase-3/prob-2/raw_train.parquet'
train_raw_data_22 = '../../data/raw_data/phase-2/prob-2/raw_train.parquet'
link_y_data_32 = '../../data/train_data/phase-3/prob-2/train_y.parquet'
link_train_data_32 = '../../data/train_data/phase-3/prob-2/train_x.parquet'
link_y_data_22 = '../../data/train_data/phase-2/prob-2/train_y.parquet'
link_train_data_22 = '../../data/train_data/phase-2/prob-2/train_x.parquet'


In [44]:
def combine_parquet_test(url_test):
    raw_test = pd.DataFrame()
    count_ = 0
    for file_path in os.listdir(url_test):
        if 'parquet' not in file_path or "123.parquet" in file_path:
            continue
        captured_data = pd.read_parquet(f'{url_test}{file_path}')
        captured_data['parquet_idx'] = count_
        raw_test = pd.concat([raw_test, captured_data])
        count_ += 1
    return raw_test

In [45]:
raw_test_32 = combine_parquet_test(link_data_test_raw_32)

In [46]:
raw_train_22 = pd.read_parquet(train_raw_data_22)
raw_train_32 = pd.read_parquet(train_raw_data_32)
order_cols = raw_train_32.columns.tolist()
order_cols.remove('label')
raw_test_32 = raw_test_32[order_cols].reset_index(drop=True)
# captured_data = captured_data[order_cols].reset_index(drop=True)

In [47]:
raw_train_22

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,label
0,0.041847,tcp,-,FIN,38.0,40.0,2438.0,19266.0,31.0,29.0,4.538438e+05,3.591177e+06,7.0,13.0,1.153722,1.058410,68.764188,66.421092,255.0,3.898436e+09,1.827204e+09,255.0,0.000707,0.000566,0.000141,64.0,482.0,0.0,0.0,6.0,0.0,5.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,11.0,0.0,Normal
1,1.089133,tcp,http,FIN,14.0,18.0,1684.0,10168.0,31.0,29.0,1.148804e+04,7.054418e+04,3.0,5.0,83.751772,64.035706,9346.434820,8182.385202,255.0,3.051186e+09,9.067852e+08,255.0,0.000665,0.000523,0.000142,120.0,565.0,1.0,3924.0,1.0,0.0,2.0,1.0,1.0,2.0,0.0,0.0,1.0,2.0,1.0,0.0,Normal
2,0.000002,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,2.280000e+08,0.000000e+00,0.0,0.0,0.002000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,25.0,2.0,18.0,17.0,17.0,25.0,0.0,0.0,0.0,17.0,25.0,0.0,Other
3,1.467246,tcp,ftp,FIN,12.0,12.0,2618.0,682.0,254.0,252.0,1.308574e+04,3.413197e+03,3.0,4.0,133.386003,124.152453,7744.976658,198.329344,255.0,2.477915e+09,1.653923e+09,255.0,0.173821,0.101319,0.072502,218.0,57.0,0.0,0.0,3.0,1.0,1.0,1.0,1.0,3.0,0.0,0.0,0.0,2.0,3.0,0.0,Denial of Service
4,0.000927,udp,dns,CON,2.0,2.0,130.0,162.0,31.0,29.0,5.609493e+05,6.990291e+05,0.0,0.0,0.002000,0.003000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,65.0,81.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,2.0,0.0,0.0,0.0,1.0,4.0,0.0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61836,1.751882,tcp,smtp,FIN,52.0,40.0,37232.0,3276.0,31.0,29.0,1.667510e+05,1.459002e+04,18.0,8.0,34.343178,44.907129,2566.039901,81.466320,255.0,2.862144e+09,7.879814e+08,255.0,0.000631,0.000497,0.000134,716.0,82.0,0.0,0.0,2.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,Normal
61837,0.929489,tcp,-,FIN,10.0,10.0,608.0,646.0,254.0,252.0,4.716570e+03,5.009204e+03,1.0,4.0,6535.779900,6535.937500,644277.300300,19332.784000,255.0,1.045046e+09,3.929620e+09,255.0,0.206274,0.095900,0.110374,61.0,65.0,0.0,0.0,3.0,1.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,3.0,3.0,0.0,Exploits
61838,0.000008,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,5.700000e+07,0.000000e+00,0.0,0.0,0.008000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,51.0,2.0,41.0,41.0,20.0,51.0,0.0,0.0,0.0,41.0,51.0,0.0,Other
61839,0.000009,unas,-,INT,2.0,0.0,200.0,0.0,254.0,0.0,8.888889e+07,0.000000e+00,0.0,0.0,0.009000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,100.0,0.0,0.0,0.0,5.0,2.0,1.0,1.0,1.0,5.0,0.0,0.0,0.0,3.0,5.0,0.0,Information Gathering


In [10]:
raw_train_32

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,label
0,0.000008,ptp,-,INT,2.0,0.0,200.0,0.0,254.0,0.0,1.000000e+08,0.000000e+00,0.0,0.0,0.008000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,100.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,26.0,2.0,0.0,Exploits
1,0.778566,tcp,-,FIN,20.0,36.0,1096.0,28700.0,62.0,252.0,1.070686e+04,2.867117e+05,6.0,16.0,40.977159,21.695715,2279.242426,1794.156724,255.0,4.028057e+09,2.754592e+09,255.0,0.060286,0.015325,0.044961,55.0,797.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,Exploits
2,0.054953,tcp,-,FIN,84.0,86.0,4862.0,77276.0,31.0,29.0,6.995068e+05,1.111903e+07,7.0,36.0,0.658349,0.640859,0.000000,42.532364,255.0,1.060385e+08,1.285389e+08,255.0,0.000613,0.000476,0.000137,58.0,899.0,0.0,0.0,10.0,0.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,5.0,0.0,Normal
3,1.521637,tcp,-,FIN,20.0,16.0,1020.0,1242.0,62.0,252.0,5.094514e+03,6.124982e+03,5.0,7.0,80.034997,101.031135,4123.153783,5894.634853,255.0,1.116041e+09,1.639013e+09,255.0,0.052784,0.006159,0.046625,51.0,78.0,0.0,0.0,3.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,Exploits
4,0.000004,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,1.140000e+08,0.000000e+00,0.0,0.0,0.004000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,31.0,2.0,7.0,7.0,4.0,31.0,0.0,0.0,0.0,7.0,31.0,0.0,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92757,0.000003,unas,-,INT,2.0,0.0,200.0,0.0,254.0,0.0,2.666667e+08,0.000000e+00,0.0,0.0,0.003000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,100.0,0.0,0.0,0.0,9.0,2.0,4.0,4.0,4.0,9.0,0.0,0.0,0.0,13.0,9.0,0.0,Information Gathering
92758,0.000002,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,2.280000e+08,0.000000e+00,0.0,0.0,0.002000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,27.0,2.0,11.0,11.0,11.0,27.0,0.0,0.0,0.0,11.0,27.0,0.0,Other
92759,58.795750,ospf,-,REQ,26.0,0.0,2808.0,0.0,254.0,0.0,3.673735e+02,0.000000e+00,0.0,0.0,2462.994500,0.000000,2571.657000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,108.0,0.0,0.0,0.0,1.0,6.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,Exploits
92760,0.412985,tcp,http,FIN,12.0,10.0,1326.0,2174.0,62.0,252.0,2.355534e+04,3.790937e+04,3.0,3.0,37.544092,44.214000,1877.802520,57.948395,255.0,3.991123e+09,2.994732e+09,255.0,0.061542,0.014241,0.047301,111.0,217.0,1.0,321.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,Exploits


In [11]:
os.environ.get("MLFLOW_TRACKING_URI")

'http://localhost:5000'

In [12]:
# mlflow_evn = os.environ.get("MLFLOW_TRACKING_URI")
mlflow_evn = "http://localhost:5000"
mlflow.set_tracking_uri(mlflow_evn)

In [16]:
model_uri = os.path.join(
    "models:/", "phase-2_prob-2_lgbm__", "6"
)
input_schema = mlflow.models.Model.load(model_uri).get_input_schema().to_dict()
model = mlflow.sklearn.load_model(model_uri)
get_features = [each['name'] for each in input_schema]

In [50]:
train_data_32 = pd.read_parquet(link_train_data_32)
train_data_22 = pd.read_parquet(link_train_data_22)
y_train_data_32 = pd.read_parquet(link_y_data_32)

In [51]:
pd.concat([train_data_32, train_data_22[train_data_32.columns.tolist()]])

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41
0,0.022604,112,0,3,60.0,62.0,3614.0,45696.0,31.0,29.0,1.257830e+06,1.591187e+07,7.0,24.0,0.383879,0.360836,0.000000,19.111911,255.0,1.081195e+08,2.270011e+09,255.0,0.000732,0.000590,0.000142,60.0,737.0,0.0,0.0,20.0,0.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,4.0,8.0,0.0
1,3.596744,112,0,3,14.0,26.0,648.0,22436.0,62.0,252.0,1.338989e+03,4.798562e+04,3.0,11.0,276.672606,138.899084,24169.387720,16434.218170,255.0,2.161779e+09,4.284151e+09,255.0,0.210706,0.110831,0.099875,46.0,863.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0
2,17.574184,112,0,3,78.0,82.0,4550.0,72914.0,31.0,29.0,2.044818e+03,3.278673e+04,7.0,33.0,231.234717,216.956747,45300.017650,44176.942600,255.0,2.189063e+09,2.189415e+09,255.0,0.000809,0.000673,0.000136,58.0,889.0,0.0,0.0,3.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,7.0,0.0
3,0.000006,118,0,4,2.0,0.0,104.0,0.0,254.0,0.0,6.933333e+07,0.000000e+00,0.0,0.0,0.006000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,52.0,0.0,0.0,0.0,8.0,2.0,2.0,1.0,1.0,7.0,0.0,0.0,0.0,18.0,7.0,0.0
4,0.511935,112,0,3,16.0,10.0,900.0,556.0,254.0,252.0,1.318917e+04,7.829119e+03,5.0,3.0,32.660732,52.016223,1676.521258,66.212078,255.0,2.751382e+09,4.239611e+09,255.0,0.078317,0.043780,0.034537,56.0,56.0,0.0,0.0,5.0,1.0,1.0,1.0,1.0,5.0,0.0,0.0,0.0,1.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34278,0.024829,112,9,3,52.0,40.0,37492.0,3276.0,31.0,29.0,1.184776e+07,1.029441e+06,18.0,8.0,0.489700,0.623615,30.149826,1.113257,255.0,2.466449e+09,3.220780e+08,255.0,0.000627,0.000504,0.000123,721.0,82.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,4.0,2.0,0.0
34279,1.838529,112,0,3,20.0,18.0,9240.0,9108.0,254.0,252.0,3.819575e+04,3.742992e+04,7.0,6.0,96.720632,96.779473,5622.977390,5909.666723,255.0,1.319023e+09,3.517653e+09,255.0,0.274392,0.193268,0.081124,462.0,506.0,0.0,0.0,3.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0
34280,0.000003,118,2,4,2.0,0.0,114.0,0.0,254.0,0.0,1.520000e+08,0.000000e+00,0.0,0.0,0.003000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,23.0,2.0,17.0,16.0,16.0,23.0,0.0,0.0,0.0,17.0,23.0,0.0
34281,0.000007,90,0,4,2.0,0.0,200.0,0.0,254.0,0.0,1.142857e+08,0.000000e+00,0.0,0.0,0.007000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,100.0,0.0,0.0,0.0,4.0,2.0,2.0,2.0,2.0,4.0,0.0,0.0,0.0,2.0,4.0,0.0


In [37]:
preds = model.predict(train_data_32)

In [38]:
y_train_data_32['pred'] = preds

In [39]:
y_train_data_32[y_train_data_32['pred'] == y_train_data_32['label']].shape[0] / y_train_data_32.shape[0]

0.8637962054301603

In [46]:
#feature 22, 19 -> convert to binary (0, 255) và null
#feature 10 -> (0, 252, 29, 60) và null
#feature 9
raw_train["feature9"].value_counts()

feature9
254.0    60740
31.0     20215
62.0     10113
0.0       1540
64.0        72
1.0         31
29.0        14
63.0        12
60.0        12
255.0        8
32.0         4
30.0         1
Name: count, dtype: int64

In [47]:
raw_test["feature9"].value_counts()

feature9
254.0    73316
31.0     23998
62.0     11977
0.0       1815
64.0        90
63.0        36
29.0        18
1.0         18
255.0       18
32.0        17
60.0         9
Name: count, dtype: int64

In [95]:
def data_processing_phase2_prob2(data, phase="train"):
    drop_corr = ['feature5', 'feature13', 'feature6', 'feature14', 'feature22', 'feature30', 'feature35', 'feature32', 'feature36', 'feature37']
    convert_cate_cols = ['feature9', 'feature10', 'feature31']
    data = data.drop(columns = drop_corr)
    for col in convert_cate_cols:
        data[col] = data[col].astype(int).astype(str)
    cate_cols = ['feature2', 'feature3', 'feature4']
    for col in cate_cols:
        dict_count = data[col].value_counts().to_dict()
        list_below = [key for key, value in dict_count.items() if value / data.shape[0] < 0.0005]
        if phase == 'test':
            list_below.append('null')
        data[col] = data[col].replace({value: 'other' for value in list_below})
    return data

In [106]:
[each for each in raw_test_['feature4'].unique() if each not in each for each in raw_train_['feature4'].unique()]

[]

In [107]:
def get_num(n):
    return n

dict_ = {}
dict_[1] = get_num

In [109]:
dict_[1](10)

10

In [111]:
import pickle
with open("../../data/train_data/phase-2/prob-2/category_index.pickle", "rb") as file:
    category_features = pickle.load(file)

In [112]:
category_features 

{'feature2': Index(['arp', 'ospf', 'other', 'sctp', 'tcp', 'udp', 'unas'], dtype='object'),
 'feature3': Index(['-', 'dhcp', 'dns', 'ftp', 'ftp-data', 'http', 'other', 'pop3', 'smtp',
        'ssh', 'ssl'],
       dtype='object'),
 'feature4': Index(['CON', 'FIN', 'INT', 'REQ', 'other'], dtype='object')}